<a href="https://colab.research.google.com/github/HemaGarima/Machine-Learning/blob/master/Siamese_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Simese model using Trax

In [1]:
pip install trax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 36.5 MB/s eta 0:00:00


In [2]:
import trax
from trax import layers as tl
import trax.fastmath.numpy as np
import numpy

# setting rangom seeds
numpy.random.seed(10)

## L2 Normalization

- Before building the model you will need to define a function that applies L2 normalization to a tensor.

In [3]:
def normalize(x):
  return x/np.sqrt(np.sum(x*x, axis = -1, keepdims = True))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


- Notice that the denominator can be replaced by np.linalg.norm(x,axis = -1 , keepdims = True) to achieve the same results and that Trax's numpy is being used within the function.

In [4]:
tensor = numpy.random.random((2,5))
print(f'The normalized tensor is of type: {type(tensor)}\n\nAnd looks like this:\n\n {tensor}')

The normalized tensor is of type: <class 'numpy.ndarray'>

And looks like this:

 [[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701]
 [0.22479665 0.19806286 0.76053071 0.16911084 0.08833981]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
norm_tensor = normalize(tensor)
print(f'The normalized tensor is of type: {type(norm_tensor)}\n\n looks like this:\n\n {norm_tensor}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The normalized tensor is of type: <class 'jaxlib.xla_extension.ArrayImpl'>

 looks like this:

 [[0.5739379  0.01544148 0.4714962  0.5571832  0.37093794]
 [0.26781026 0.23596111 0.9060541  0.20146926 0.10524315]]


- Notice that the initial tensor was converted from a numpy array to a jax array in the process.

## Siamese Model

### To create a Siamese model you first need to create a LSTM model using the Serial combinator and then use another combinator layer called Paralled to create the Siamese model. You should be familiar with the following layers:

- Serial : A combinator layer that allows to stack layers serially using function composition.

- Embedding : Maps descrete tokens to vectors. It will have shape (vocabulary length X dimension of output vectors). The dimension of output vectors (also called d_feature) is the number of elements in the word embedding.

- LSTM : The LSTM layer. It leverages another Trax layer called LSTMCell. The number of units should be specified and should match the number of elements in the word embedding.

- Mean : Computes the mean across a desired axis. Mean uses one tensor axis to form groups of values and replaces each group with the mean value of that group.

- Fn : Layer with no weights that applies the function f, which should be specified using a lambda syntax.

- Parallel : It is a combinator layer (like Serial) that applies a list of layers in parallel to its inputs.

In [10]:
vocab_size = 500
model_dimension  = 128

# Define the LSTM model
LSTM = tl.Serial(
    tl.Embedding(vocab_size = vocab_size , d_feature = model_dimension),
    tl.LSTM(model_dimension),
    tl.Mean(axis = 1),
    tl.Fn('Normalize',lambda x : normalize(x))
)

# Use the Parallel combinator to create a Siamese model out of the LSTM
Siamese = tl.Parallel(LSTM , LSTM)

Next is a helper function that prints information for every layer (sublayer within Serial):

In [13]:
def show_layers(model , layer_prefix):
  print(f"Total layers : {len(model.sublayers)}\n")
  for i in range(len(model.sublayers)):
    print('========')
    print(f'{layer_prefix}_{i}: {model.sublayers[i]}\n')

print('Siamese model:\n')
show_layers(Siamese , 'Parallel.sublayers')

print('Detail of LSTM models : \n')
show_layers(LSTM , 'Serial.sublayers')

Siamese model:

Total layers : 2

Parallel.sublayers_0: Serial[
  Embedding_500_128
  LSTM_128
  Mean
  Normalize
]

Parallel.sublayers_1: Serial[
  Embedding_500_128
  LSTM_128
  Mean
  Normalize
]

Detail of LSTM models : 

Total layers : 4

Serial.sublayers_0: Embedding_500_128

Serial.sublayers_1: LSTM_128

Serial.sublayers_2: Mean

Serial.sublayers_3: Normalize

